In [20]:
import os

from keras import Input
from keras import backend as K
from keras.layers import Conv2D, MaxPooling2D, Lambda, Flatten, Dense
from keras.models import Sequential, save_model, load_model
from keras.models import Model
from keras.regularizers import l2
from keras.optimizers import Adam

import numpy as np
from PIL import Image, ImageFilter

from file_handler import handler

In [21]:
image_example = 'resources/proc_data/train_data/false-0.jpg'

img_width, img_height = handler.get_image_size(image_example)

input_shape = (img_height, img_width, 1)

In [22]:
def image_preparation(img: str) -> np.array:
    img = Image.open(img)
    gray_img = img.filter(ImageFilter.SMOOTH)
    edges_img = gray_img.filter(ImageFilter.FIND_EDGES)
    # emboss = edges_img.filter(ImageFilter.EMBOSS)
    # edges_img.show()
    return np.array(edges_img)

In [23]:
image_preparation('train/true/true-0.jpg')

array([[255, 255, 255, ..., 255, 255, 255],
       [255,   0,   0, ...,   0,   0, 255],
       [255,   0,   0, ...,   0,   0, 255],
       ...,
       [255,   0,   0, ...,   0,   0, 255],
       [255,   0,   0, ...,   0,   0, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)

In [24]:
true_img_0 = 'train/true/true-0.jpg'
true_img_1 = 'train/true/true-1.jpg'
true_img_2 = 'train/true/true-2.jpg'
true_img_3 = 'train/true/true-3.jpg'
true_img_4 = 'train/true/true-4.jpg'
true_img_5 = 'train/true/true-5.jpg'
true_img_6 = 'train/true/true-6.jpg'
true_img_7 = 'train/true/true-7.jpg'
true_img_8 = 'train/true/true-8.jpg'
true_img_9 = 'train/true/true-9.jpg'
true_img_10 = 'train/true/true-10.jpg'
true_img_11 = 'train/true/true-11.jpg'

false_img_0 = 'train/false/false-0.jpg'
false_img_1 = 'train/false/false-1.jpg'
false_img_2 = 'train/false/false-2.jpg'
false_img_3 = 'train/false/false-3.jpg'


val_img_0 = 'val/true/true-1.jpg'
val_img_1 = 'val/true/true-6.jpg'
val_img_2 = 'val/true/true-0.jpg'
val_img_3 = 'val/false/false-6.jpg'

X_train = np.array(
    [
        [image_preparation(img=true_img_0), image_preparation(img=true_img_1)],
        [image_preparation(img=true_img_2), image_preparation(img=false_img_0)],
        [image_preparation(img=true_img_4), image_preparation(img=true_img_5)],
        [image_preparation(img=true_img_6), image_preparation(img=false_img_2)],
        [image_preparation(img=true_img_8), image_preparation(img=true_img_9)],
        [image_preparation(img=true_img_10), image_preparation(img=true_img_11)],
        [image_preparation(img=true_img_3), image_preparation(img=false_img_1)],
        [image_preparation(img=true_img_7), image_preparation(img=false_img_3)]
    ]
)

X_val = np.array([[image_preparation(val_img_0), image_preparation(val_img_1)],[image_preparation(img=val_img_2), image_preparation(img=val_img_3)]])
Y_train = np.array((1, 0, 1, 0, 1, 0, 1, 0))
Y_val = np.array((1, 0))

In [25]:
X_train = X_train.reshape((8, 2, 105, 605, 1))
X_val = X_val.reshape((2, 2, 105, 605, 1))

In [26]:
def initialize_weights(shape, dtype=None):
  return np.random.normal(loc = 0.0, scale = 1e-2, size = shape)

def initialize_bias(shape, dtype=None):
  return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)

In [27]:
def siamese_nn(input_shape):
    """
        Model architecture
    """

    # Define the tensors for the two input images
    left_input = Input(input_shape)
    right_input = Input(input_shape)

    # Convolutional Neural Network
    model = Sequential()
    model.add(Conv2D(64, (10,10), activation='relu', input_shape=input_shape,
                   kernel_initializer=initialize_weights,
                   bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (7,7), activation='relu',
                     kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (2,2), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(256, (2,2), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(Flatten())
    model.add(Dense(1024, activation='sigmoid',
                   kernel_regularizer=l2(2e-3),
                   kernel_initializer=initialize_weights,bias_initializer=initialize_bias))

    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)

    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])

    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1,activation='sigmoid',bias_initializer=initialize_bias)(L1_distance)

    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)

    # return the model
    return siamese_net

In [58]:
model = siamese_nn(input_shape)
model.summary()
optimizer = Adam(learning_rate=0.00001)
model.compile(loss=['poisson'], optimizer=optimizer, metrics=['accuracy'])
history_cnn = model.fit(
    [X_train[:,0], X_train[:,1]],
    Y_train,
    batch_size=16,
    epochs=8,
    validation_data=([X_val[:,0], X_val[:,1]], Y_val)
  )

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 105, 605, 1  0           []                               
                                )]                                                                
                                                                                                  
 input_12 (InputLayer)          [(None, 105, 605, 1  0           []                               
                                )]                                                                
                                                                                                  
 sequential_5 (Sequential)      (None, 1024)         168116032   ['input_11[0][0]',               
                                                                  'input_12[0][0]']         

In [61]:
test_img_0 = 'train/true/true-0.jpg'
test_img_1 = 'train/true/true-8.jpg'
test_img_2 = 'val/true/true-0.jpg'
test_img_3 = 'test/false/false-0.jpg'
x_test = np.array([[image_preparation(test_img_0), image_preparation(test_img_2)]]).reshape((1, 2, 105, 605, 1))
y_test = np.array([0])

In [62]:
model.predict([x_test[:,0],x_test[:,1]])

1/1 [==============================] - 0s 85ms/step


array([[0.522997]], dtype=float32)

In [57]:
model.evaluate([x_test[:,0],x_test[:,1]], y_test)

1/1 [==============================] - 0s 109ms/step - loss: 17.4066 - accuracy: 0.0000e+00


[17.406583786010742, 0.0]